In [2]:
# Data wrangling
import pandas as pd
import numpy as np
# Data visualization
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
# Off FutureWarnings
import warnings 
warnings.filterwarnings('ignore')
#Resampling
from imblearn.over_sampling import SMOTENC 
from sklearn.utils import class_weight
#Dimension Reduction
from sklearn.decomposition import PCA
# Preprocessing
from sklearn.preprocessing import StandardScaler, PowerTransformer, OrdinalEncoder, OneHotEncoder 
from sklearn.preprocessing import LabelEncoder
# Models
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
# Models Pipelines
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
# Model evaluation
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
# Save model
import pickle

In [3]:
## Upload df
df = pd.read_csv('cleaned_data.csv') 
df

,Unnamed: 0,Type,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Machine_failure,TWF,HDF,PWF,OSF,RNF,Failure_type
0,0,Medium,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,NF
1,1,Low,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,NF
2,2,Low,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,NF
3,3,Low,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,NF
4,4,Low,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,NF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,9995,Medium,298.8,308.4,1604,29.5,14,0,0,0,0,0,0,NF
9972,9996,High,298.9,308.4,1632,31.8,17,0,0,0,0,0,0,NF
9973,9997,Medium,299.0,308.6,1645,33.4,22,0,0,0,0,0,0,NF
9974,9998,High,299.0,308.7,1408,48.5,25,0,0,0,0,0,0,NF


In [4]:
from sklearn.metrics import f1_score 

def get_metrics(y_true, y_pred, unique_classes):
    # Calculating F1 scores for each class
    f1_scores_per_class = f1_score(y_true, y_pred, average=None, labels=unique_classes)
    recall_scores_per_class = recall_score(y_true, y_pred, average=None, labels=unique_classes)
    precision_scores_per_class = precision_score(y_true, y_pred, average=None, labels=unique_classes)
    class_f1_scores = dict(zip(unique_classes, f1_scores_per_class))
    class_recall_scores = dict(zip(unique_classes, recall_scores_per_class))
    class_precision_scores = dict(zip(unique_classes, precision_scores_per_class))
    dict_metrics = {
    'Accuracy': accuracy_score(y_true, y_pred),
    'Balanced Accuracy': balanced_accuracy_score(y_true, y_pred),
    'Macro Recall': recall_score(y_true, y_pred, average='macro'), 
    'Macro Precision': precision_score(y_true, y_pred, average='macro'), 
    'Macro F1': f1_score(y_true, y_pred, average='macro'),
    'F1 Scores per Class': class_f1_scores,
    'Recall Scores per Class': class_recall_scores,
    'Precision Scores per Class': class_precision_scores
    }
    return dict_metrics

In [5]:
df

,Unnamed: 0,Type,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Machine_failure,TWF,HDF,PWF,OSF,RNF,Failure_type
0,0,Medium,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,NF
1,1,Low,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,NF
2,2,Low,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,NF
3,3,Low,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,NF
4,4,Low,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,NF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,9995,Medium,298.8,308.4,1604,29.5,14,0,0,0,0,0,0,NF
9972,9996,High,298.9,308.4,1632,31.8,17,0,0,0,0,0,0,NF
9973,9997,Medium,299.0,308.6,1645,33.4,22,0,0,0,0,0,0,NF
9974,9998,High,299.0,308.7,1408,48.5,25,0,0,0,0,0,0,NF


In [6]:
NUMERIC_FEATURES = ['Air_temperature', 'Process_temperature', 'Rotational_speed', 'Torque', 'Tool_wear']
CATEGORIC_FEATURES = ['Type']

In [7]:
# Create preprocessor ColumnTransformer to do OneHotEncoder for CATEGORIC_FEATURES and StandardScaler() for NUMERIC_FEATURES
# Define the pipelines for numeric and categorical transformations
num_pipeline = Pipeline([
    ('num_features', StandardScaler()) 
    ])
cat_pipeline = Pipeline([ 
    ('cat_features', OneHotEncoder())
    ])
# Create the ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num_trans', num_pipeline, NUMERIC_FEATURES),
    ('cat_trans', cat_pipeline, CATEGORIC_FEATURES) ])
# Fit and transform the data
df_transformed = preprocessor.fit_transform(df)
# Converting the transformed data back to a dataframe for easier visualization
# The transformed data will have new column names, especially for the one hot encoded categories 
encoded_feature_names = preprocessor.named_transformers_['cat_trans'].get_feature_names_out(CATEGORIC_FEATURES) 
new_column_names = list(NUMERIC_FEATURES) + list(encoded_feature_names)
df_transformed = pd.DataFrame(df_transformed, columns=new_column_names)
df_transformed.head()


,Air_temperature,Process_temperature,Rotational_speed,Torque,Tool_wear,Type_High,Type_Low,Type_Medium
0,-0.951551,-0.946692,0.065483,0.289789,-1.695147,0.0,0.0,1.0
1,-0.901538,-0.879314,-0.732576,0.643119,-1.647949,0.0,1.0,0.0
2,-0.951551,-1.014071,-0.230301,0.956069,-1.616484,0.0,1.0,0.0
3,-0.901538,-0.946692,-0.593055,-0.043351,-1.585019,0.0,1.0,0.0
4,-0.901538,-0.879314,-0.732576,0.007125,-1.553553,0.0,1.0,0.0


In [8]:
# df = df[df['Failure_type'] != 'TWF']

In [9]:
df_model = df.copy()
X = df_model[NUMERIC_FEATURES + CATEGORIC_FEATURES]
y = df['Failure_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

In [10]:
from sklearn.neural_network import MLPClassifier

In [11]:
unique_classes = np.unique(y_train)
unique_classes

array(['HDF', 'NF', 'OSF', 'PWF', 'TWF'], dtype=object)

In [13]:
# Creating pipeline with PCA analysis and balanced class 
pip_model_no_pca = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', MLPClassifier(random_state=2023))
    ])
# Fit pipeline with PCA
# weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)
pip_model_no_pca.fit(X_train, y_train)
# Generate Predictions using the correctly fitted pipeline 
y_pred = pip_model_no_pca.predict(X_test)
# Evaluate Metrics
metrics = get_metrics(y_test, y_pred, unique_classes)
# View Results
metrics

{'Accuracy': 0.9879759519038076,
 'Balanced Accuracy': 0.7204983388704319,
 'Macro Recall': 0.7204983388704319,
 'Macro Precision': 0.701796650935209,
 'Macro F1': 0.7109004400692067,
 'F1 Scores per Class': {'HDF': 0.8372093023255814,
  'NF': 0.9938080495356038,
  'OSF': 0.875,
  'PWF': 0.8484848484848485,
  'TWF': 0.0},
 'Recall Scores per Class': {'HDF': 0.8571428571428571,
  'NF': 0.9953488372093023,
  'OSF': 0.875,
  'PWF': 0.875,
  'TWF': 0.0},
 'Precision Scores per Class': {'HDF': 0.8181818181818182,
  'NF': 0.9922720247295209,
  'OSF': 0.875,
  'PWF': 0.8235294117647058,
  'TWF': 0.0}}

In [13]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [16]:
import warnings 
warnings.filterwarnings('ignore')

In [19]:
fine_tune_params = {
    'model__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'model__alpha': [0.0001, 0.001, 0.01, 1, 10, 100],
    'model__learning_rate': ['constant', 'adaptive'],
}
# Running a new GridSearchCV for fine-tuning
fine_tune_grid = GridSearchCV(pip_model_no_pca, fine_tune_params, cv=3, scoring='f1_macro', n_jobs=-1, verbose=1)
fine_tune_grid.fit(X_train, y_train_encoded)

# Collecting and printing the fine-tuned results
fine_tuned_results = pd.DataFrame(fine_tune_grid.cv_results_) 
fine_tuned_best_index = fine_tuned_results['mean_test_score'].idxmax()
fine_tuned_best_params = fine_tuned_results.loc[fine_tuned_best_index, 'params']

# Print best model parameters
print("Best fine-tuned model parameters:") 
print(fine_tuned_best_params)
# Finding the best estimator paramaters 
tuned_model = fine_tune_grid.best_estimator_ 
y_pred_encoded = tuned_model.predict(X_test)
y_pred = label_encoder.inverse_transform(y_pred_encoded)
# View new perfomance (focus on F1-score) 
get_metrics(y_test, y_pred, unique_classes)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/Users/mihirthalanki/anaconda3/envs/e4_predictive_maintenance/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mihirthalanki/anaconda3/envs/e4_predictive_maintenance/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mihirthalanki/anaconda3/envs/e4_predictive_maintenance/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mihirthalanki/anaconda3/envs/e4_predictive_maintenance/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarn

Best fine-tuned model parameters:
{'model__alpha': 0.01, 'model__hidden_layer_sizes': (50, 50), 'model__learning_rate': 'constant'}


{'Accuracy': 0.9884769539078156,
 'Balanced Accuracy': 0.7359745293466224,
 'Macro Recall': 0.7359745293466224,
 'Macro Precision': 0.7018900343642612,
 'Macro F1': 0.71760773823298,
 'F1 Scores per Class': {'HDF': 0.8292682926829268,
  'NF': 0.9940645161290322,
  'OSF': 0.8823529411764706,
  'PWF': 0.8823529411764706,
  'TWF': 0.0},
 'Recall Scores per Class': {'HDF': 0.8095238095238095,
  'NF': 0.9953488372093023,
  'OSF': 0.9375,
  'PWF': 0.9375,
  'TWF': 0.0},
 'Precision Scores per Class': {'HDF': 0.85,
  'NF': 0.9927835051546392,
  'OSF': 0.8333333333333334,
  'PWF': 0.8333333333333334,
  'TWF': 0.0}}

In [20]:
y_pred_encoded = tuned_model.predict(X_train)
y_pred = label_encoder.inverse_transform(y_pred_encoded)
# View new perfomance (focus on F1-score) 
get_metrics(y_train, y_pred, unique_classes)

{'Accuracy': 0.9939849624060151,
 'Balanced Accuracy': 0.7847167905623788,
 'Macro Recall': 0.7847167905623788,
 'Macro Precision': 0.9959750276495111,
 'Macro F1': 0.825350514605271,
 'F1 Scores per Class': {'HDF': 0.8974358974358975,
  'NF': 0.9969064191802011,
  'OSF': 1.0,
  'PWF': 0.976,
  'TWF': 0.2564102564102564},
 'Recall Scores per Class': {'HDF': 0.8235294117647058,
  'NF': 0.9998707175177763,
  'OSF': 1.0,
  'PWF': 0.953125,
  'TWF': 0.14705882352941177},
 'Precision Scores per Class': {'HDF': 0.9859154929577465,
  'NF': 0.9939596452898085,
  'OSF': 1.0,
  'PWF': 1.0,
  'TWF': 1.0}}